In [1]:
# change the directory
import os 
from dotenv import load_dotenv
from typing import TypedDict, List, Annotated
from operator import add

# import relevant packages
os.chdir(os.getenv('WORKSPACE_DIRECTORY'))
from agents.base_llm import OpenAILLMs
from agents.master import MasterWorkflow
from agents.user import UserLLM
from agents.diagnosis import DiagnosisLLM

from pdf_database.vector_database import PDFVectorDatabase

# from agents.diagnosis_team.diagnosis_coordinator import DiagnosticCoordinator
# from agents.diagnosis_team.generalist import GeneralistLLM

#### Mocked Data

In [3]:
class Report(TypedDict):
    """
    Defines the state for the multi-agent workflow
    Includes messages, patient input, and workflow states
    """
    patient_input: str
    messages: Annotated[List[dict], add]
    diagnosis: str
    recommendations: str
    appointment_details: str

mocked_report_empty : Report = {'patient_input': '',
                          'messages' : [],
                          'diagnosis' : '',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mocked_report_patient_filled : Report = {'patient_input': 'The user is a 45-year-old male named John Doe who is experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension managed with medication and borderline high cholesterol with no current treatment.',
                          'messages' : [],
                          'diagnosis' : '',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mocked_report_diagnosis_filled : Report = {'patient_input': 'The user is a 45-year-old male named John Doe who is experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension managed with medication and borderline high cholesterol with no current treatment.',
                          'messages' : [],
                          'diagnosis' : 'The possible diagnoses for John Doe include acute coronary syndrome, stable angina, heart failure, hypertensive heart disease, pulmonary embolism, gastroesophageal reflux disease, or anxiety/panic attack, and further tests are needed to confirm the diagnosis.',
                          'recommendations' : '',
                          'appointment_details' : '',
}

mock_patient_info = """
John Doe, a 45-year-old male, reports experiencing persistent chest pain, shortness of breath, occasional dizziness, and fatigue for the past two weeks. His medical history includes hypertension diagnosed five years ago, managed with medication, and borderline high cholesterol with no current treatment. He has a family history of heart disease, as his father suffered a heart attack at age 50. He has no known allergies.
"""

pdf_directory_one = 'pdf_database\diagnosis_pdfs'

#### Test Base LLMs

In [ ]:
# PASS
def test_base_llm():
    test_agent = OpenAILLMs()
    print(test_agent.prompt)
    test_agent_result = test_agent("What is the weather today")
    return test_agent_result
# print(test_base_llm())

#### Test Master Delegation

In [ ]:
# PASS : USER INTERACTION
def test_master_default():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report_empty)
    return output
# print(test_master_default())

# PASS : DIAGNOSIS
def test_master_diagnosis():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report_patient_filled)
    return output
# print(test_master_delegation())

# PASS : RECOMMENDATION
def test_master_recommendation():
    master_agent = MasterWorkflow()
    output = master_agent.test_call(mocked_report_diagnosis_filled)
    return output
# print(test_master_recommendation())



#### Test User Agent and Tools

In [ ]:
# PASS
def test_user():
    "my symptoms are shortness of breath and acute chest pain"
    "32"
    "i do not have any medical history"
    user_agent = UserLLM()
    output = user_agent()
    return output
# print(test_user())

#### Test Diagnosis Agent and Tools

In [ ]:
# PASS
def test_diagnose():
    diagnosis_agent = DiagnosisLLM()
    output = diagnosis_agent(mocked_report_patient_filled)
    return output
# print(test_diagnose())

#### Test Synergies and Workflow

In [ ]:
# STEP ONE PASS : MASTERS TO USER
def master_to_user():
    master_agent = MasterWorkflow()
    output = master_agent.test_run(mocked_report_empty)
    return output
# print(master_to_user())

# STEP TWO PASS: MASTER TO USER TO DIAGNOSIS
def master_to_diagnosis():
    master_agent = MasterWorkflow()
    output = master_agent.test_run(mocked_report_patient_filled)
    return output
# print(master_to_diagnosis())

# STEP THREE : MASTER TO USER TO DIAGNOSIS TO RECOMMENDATION
def master_to_recommendation():
    return None


#### Test Tools

In [4]:
# test vector database
def test_vector_database():
    # PASS
    vector_database = PDFVectorDatabase()
    # PASS
    vector_database.add_pdf_directory(pdf_directory_one)
    # PASS
    vector_database.load_pdfs()
    # PASS
    vector_database.build_database()
    # # PASS
    output = vector_database.search_with_metadata(query="something is wrong with my cardiovascular")
    return output

test_vector_database()

File exists
[Document(metadata={'source': 'd_one.pdf', 'page': 0}, page_content='Citation: Rubi´ s, P .P . Cardiac Disease:\nDiagnosis, Treatment, and Outcomes.\nJ. Pers. Med.2022, 12, 1212. https://\ndoi.org/10.3390/jpm12081212\nReceived: 18 July 2022\nAccepted: 20 July 2022\nPublished: 26 July 2022\nPublisher’s Note:MDPI stays neutral\nwith regard to jurisdictional claims in\npublished maps and institutional afﬁl-\niations.\nCopyright: © 2022 by the author.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed under the terms and\nconditions of the Creative Commons\nAttribution (CC BY) license (https://\ncreativecommons.org/licenses/by/\n4.0/).\nJournal of\nPersonalized \nMedicine\nEditorial\nCardiac Disease: Diagnosis, Treatment, and Outcomes\nPaweł P . Rubi´ s\nDepartment of Cardiac and Vascular Diseases, Jagiellonian University Medical College, John Paul II Hospital,\n31-202 Krakow, Poland; pawelrub@poczta.onet.pl\nAlthough the epidemiology—as we

[Document(metadata={'source': 'd_two.pdf', 'page': 2}, page_content="You have sudden, severe weakness. \nFast or Uneven Heartbeat (Palpitations). \nIf your heart can't pump blood as well, it may beat faster to try to keep up. You may feel your heart racing or \nthrobbing. A fast or uneven heartbeat can also be the sign of an arrhythmia. This is a problem with your heart rate or \nrhythm. \n \nComplications of heart disease. \nComplications of heart disease include: \nHeart failure. One of the most common complications of heart disease, heart failure occurs when your heart can't \npump enough blood to meet your body's needs. Heart failure can result from many forms of heart disease, \nincluding heart defects, cardiovascular disease, valvular heart disease, heart infections or cardiomyopathy. \nHeart attack. A blood clot blocking the blood flow through a blood vessel that feeds the heart causes a heart \nattack, possibly damaging or destroying a part of the heart muscle. Atherosclerosis 

#### Main Workflow

In [ ]:
# # creates class 
# final_workflow = MasterWorkflow() # called once

# # starts with initial form
# initiate = final_workflow()